In [2]:
import numpy as np
import time
import random 
from copy import deepcopy
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import BCEWithLogitsLoss
from torchvision import transforms

from dataset import Cifar100
from resnet_cifar import resnet32
from utils import parameters

In [3]:
#hyperparameters
LR = parameters['LR']
WEIGHT_DECAY = parameters['WEIGHT_DECAY']
BATCH_SIZE = 512
NUM_EPOCHS = parameters['NUM_EPOCHS']
DEVICE = 'cpu'
if torch.cuda.is_available():
    DEVICE ='cuda' 
MOMENTUM=parameters['MOMENTUM']
MILESTONES=parameters['MILESTONES']
CLASSES_BATCH=parameters['CLASSES_BATCH']
NUM_CLASSES=parameters['NUM_CLASSES']
STEPDOWN_FACTOR=parameters['STEPDOWN_FACTOR']
GAMMA=parameters['GAMMA']
CRITERION=nn.BCEWithLogitsLoss()
MEMORY=2000

In [11]:
class iCaRL():
    
    def __init__(self, params=None):
        self.memory = MEMORY
        self.params = params
        self.device = DEVICE
        
        
    def train(self, net, old_net, train_dataloader, optimizer, n_epochs, n_classes):

      criterion = nn.BCEWithLogitsLoss()
      parameters_to_optimize = net.parameters()

      train_losses = []

      net.to(self.device)

      for epoch in range(n_epochs):

        if epoch in MILESTONES:
          for pg in optimizer.param_groups:
            pg['lr'] = pg['lr']/self.params['STEPDOWN_FACTOR']

        running_loss = 0.0

        for indexes, inputs, labels in train_dataloader:
          inputs = inputs.to(self.device)
          labels = labels.to(self.device)
          
          labels_hot=torch.eye(n_classes)[labels]
          labels_hot = labels_hot.to(self.device)

          net.train(True)
          # zero the parameter gradients
          optimizer.zero_grad()
          # forward
          outputs = net(inputs)

          if n_classes == 10:
            loss = criterion(outputs[:, n_classes - 10:], labels_hot[:, n_classes - 10:])
          else:
            old_outputs = self.get_old_outputs(inputs, old_net)
            targets = torch.cat((old_outputs, labels_hot[:, n_classes - 10:]), 1)
            loss = criterion(outputs, targets)

          loss.backward()
          optimizer.step()

          # statistics
          running_loss += loss.item() * inputs.size(0)

        # Calculate average losses
        epoch_loss = running_loss / float(len(train_dataloader.dataset))
        
        if epoch % 10 == 0 or epoch == (n_epochs-1):
          print('Epoch {} Loss:{:.4f}'.format(epoch, epoch_loss))
          for pg in optimizer.param_groups:
            print('Learning rate:{}'.format(pg['lr']))
          print('-'*30)

        train_losses.append(epoch_loss)

      return net, train_losses
   
    #ALGO 3
    def update_representation(self, new_data, exemplars, net, n_classes):
        '''
        X= training iamges of classes s....t
        P=(P1,....,P_s-1) #exemplars sets
        theta #current model parameters
        '''
        print('-'*30)
        print(f'**** Update Representation... ****')
        print('-'*30)
        
        # concatenate new data with set of exemplars
        if len(exemplars) != 0:
          data = new_data + exemplars
        else:
          data = new_data
        
        old_net = deepcopy(net) #salva network 
        
        loader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
        if n_classes != 10:
          # update net last layer
          net = self.update_net(net, n_classes)
          
        optimizer = torch.optim.SGD(net.parameters(), lr=self.params['LR'], momentum=self.params['MOMENTUM'], weight_decay=self.params['WEIGHT_DECAY'])
        
        net, train_losses = self.train(net, old_net, loader, optimizer, self.params['NUM_EPOCHS'], n_classes)

        return net, train_losses
    
    

### Main


In [12]:
exemplars = {}
new_exemplars = []
exemplars_as_list = []
accuracy_new = []
accuracy_all = []

In [13]:
# Define transforms for training phase
transform_train = transforms.Compose([
                                    transforms.RandomCrop(32, padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                  ])
transform_test = transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                  ])


In [15]:
net = resnet32()
icarl=iCarl
for i in range(int(NUM_CLASSES/CLASSES_BATCH)):
    print('-'*30)
    print(f'**** ITERATION {i+1} ****')
    print('-'*30)

    n_classes = (i+1)*10 #10-20-30....

    train_dataset = Cifar100(classes=range(i*10, (i + 1)*10), train=True, transform=transform_train)
    test_dataset = Cifar100(classes=range(i*10, (i + 1)*10), train=False, transform=transform_test)

    # update representation
    net, train_losses = self.update_representation(train_dataset, exemplars_as_list, net, n_classes)
    print(train_losses)
    break
    #rappresentazione plot loss
    parameters['name']='icarl_loss'
    

    # update exemplar sets
    exemplars = self.reduce_exemplar(exemplars, n_classes)

    if herding:
      new_exemplars = self.herding_exemplar(train_dataset, n_classes, net)
    else:
      new_exemplars = self.random_exemplar(train_dataset, n_classes)

    exemplars.update(new_exemplars)

    exemplars_as_list = [item for class_exemplars in exemplars.values() for item in class_exemplars]

    # compute accuracy on the new class batch
    accuracy_new.append(self.NME(test_dataset, exemplars, net, n_classes))

    # compute accuracy on all the classes seen so far
    test_dataset_sofar = Cifar100(classes=range(0, (i + 1)*10), train=False, transform=transform_test)
    accuracy_all.append(self.NME(test_dataset_sofar, exemplars, net, n_classes))

   # return accuracy_new, accuracy_all

------------------------------
**** ITERATION 1 ****
------------------------------


Extracting Dataset/cifar-100-python.tar.gz to Dataset
Files already downloaded and verified


NameError: name 'self' is not defined

In [8]:
for i in range(int(NUM_CLASSES/CLASSES_BATCH)):
    print((i+1)*10)

10
20
30
40
50
60
70
80
90
100
